# Loop Workflow Example 1

* High level approach to making a 3D model from just a bounding box (in Australia only for now). Simply run all the cells of the notebook. 
* This requires an internet connection to access remote data, for a local data version see Example 3.

In [ ]:
#if not already installed:
#!conda install -c loop3d map2loop loopstructural pyamg meshio

### Optional:
Create a config dictionary that specifies the column names and description texts to process the shape files

In [ ]:
config_dict = {
    "structure" : {
        "orientation_type": "strike",
        "dipdir_column": "strike",
        "dip_column": "dip",
        "description_column": "feature",
        "bedding_text": "Bed",
        "overturned_column": "structypei",
        "overturned_text": "BEOI",
        "objectid_column": "geopnt_id",
    },
    "geology" : {
        "unitname_column": "unitname",
        "alt_unitname_column": "code",
        "group_column": "group_",
        "supergroup_column": "supersuite",
        "description_column": "descriptn",
        "minage_column": "min_age_ma",
        "maxage_column": "max_age_ma",
        "rocktype_column": "rocktype1",
        "alt_rocktype_column": "rocktype2",
        "sill_text": "is a sill",
        "intrusive_text": "intrusive",
        "volcanic_text": "volcanic",
        "objectid_column": "objectid",
        "ignore_codes": ["cover"],
    },
    "fault" : {
        "structtype_column": "feature",
        "fault_text": "Fault",
        "dip_null_value": "0",
        "dipdir_flag": "num",
        "dipdir_column": "dip_dir",
        "dip_column": "dip",
        "orientation_type": "dip direction",
        "dipestimate_column": "dip_est",
        "dipestimate_text": "gentle,moderate,steep",
        "name_column": "name",
        "objectid_column": "objectid",
    },
    "fold" : {
        "structtype_column": "feature",
        "fold_text": "Fold axial trace",
        "description_column": "type",
        "synform_text": "syncline",
        "foldname_column": "name",
        "objectid_column": "objectid",
    }
}

## Map2Loop

In [ ]:
import time
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel, Datatype
from map2loop.sorter import SorterAlpha, SorterAgeBased, SorterUseHint, SorterUseNetworkX, SorterMaximiseContacts, SorterObservationProjections
from map2loop.thickness_calculator import ThicknessCalculatorAlpha, InterpolatedStructure, StructuralPoint
from map2loop.sampler import SamplerSpacing
from datetime import datetime

# os.environ["DOCUMENTATION_TEST"] = "True"

t0 = time.time()

# Set the region of interest for the project
bbox_3d = {
    "minx": 515687.31005864,
    "miny": 7493446.76593407,
    "maxx": 562666.860106543,
    "maxy": 7521273.57407786,
    "base": -3200,
    "top": 3000,
}

# Specify minimum details (Australian state, projection, bounding box and output file)
# Optional: the config information manually created in the config_dict can be used by
# setting the parameter config_dictionary
loop_project_filename="wa_output.loop3d"
proj = Project( 
    use_australian_state_data = "WA",
    # config_dictionary = config_dict,
    working_projection = "EPSG:28350",
    bounding_box = bbox_3d,
    verbose_level = VerboseLevel.NONE,
    loop_project_filename = loop_project_filename,
    overwrite_loopprojectfile = True
)

# Specify that the text to look for (in the unit description) to indicate a sill is not just "sill"
proj.map_data.config.update_from_dictionary({"geology": {"sill_text":"is a sill"}})

# Set the distance between sample points for arial and linestring geometry
proj.set_sampler(Datatype.GEOLOGY, SamplerSpacing(200.0))
proj.set_sampler(Datatype.FAULT, SamplerSpacing(200.0))

# Choose which thickness calculator to use
# proj.set_thickness_calculator(ThicknessCalculatorAlpha())
# proj.set_thickness_calculator(InterpolatedStructure())
proj.set_thickness_calculator(StructuralPoint())

# Choose which stratigraphic sorter to use or run_all with "take_best" flag to run them all
# proj.set_sorter(SorterAlpha())
# proj.set_sorter(SorterAgeBased())
# proj.set_sorter(SorterUseHint())
proj.set_sorter(SorterUseNetworkX())
# proj.set_sorter(SorterMaximiseContacts())
# proj.set_sorter(SorterObservationProjections())
proj.run_all()

#   or
# proj.run_all(take_best=True)

t1 = time.time()

## Extract Information from project

In [ ]:
# Save raw map data to local zipped shapefiles
proj.save_mapdata_to_files("output_files",".shp.zip")

In [ ]:
# Draw overlay of point data on geology map (can also try 'contacts','orientations','faults')
proj.draw_geology_map(overlay="contacts")

In [ ]:
# Extract map2loop estimate of the stratigraphic column
print(proj.stratigraphic_column.column)

In [ ]:
# Save geology map as a raster map in a specified projection (useful for overlays on online maps)
proj.save_geotiff_raster(filename=os.path.join(proj.map_data.tmp_path,"geol.tif"), projection="epsg:3857")

## Loop Structural

In [ ]:
import LoopProjectFile as LPF
import LoopStructural
from LoopStructural.visualisation import LavaVuModelViewer
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor

from scipy.interpolate import RegularGridInterpolator
from osgeo import gdal
import pandas as pd
import numpy as np

# LoopStructural Modelling
t2 = time.time()
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
    'nelements':1e5,  # how many tetras/voxels
    'regularisation':5,
}
projFile = LPF.ProjectFile(loop_project_filename)
processedData = LPFProcessor(projFile)
processedData.foliation_properties['sg'] = foliation_params
processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
model.nsteps=np.array([200,200,50])
model.update()


clip_on_dtm=True
if(clip_on_dtm):
    bounding_box = proj.map_data.get_bounding_box()
    minx = bounding_box['minx']
    maxx = bounding_box['maxx']
    miny = bounding_box['miny']
    maxy = bounding_box['maxy']
    model_base = bounding_box['base']
    model_top = bounding_box['top']
    dtm = gdal.Open(os.path.join("m2l_data_tmp", "StupidGDALLocalFile.tif"))
    dtm_val = dtm.GetRasterBand(1).ReadAsArray().T
    # Convert bounds to gdal raster bounds
    x = np.linspace(minx, maxx, dtm_val.shape[0])
    y = np.linspace(maxy, miny, dtm_val.shape[1])
    dtm_interpolator = RegularGridInterpolator((x,y), dtm_val)
    model.dtm = lambda xyz : dtm_interpolator(xyz[:,:2])

# Lavavu viewer
filename = os.path.join('.','vtk','surface_name_{}.vtk')
view = LavaVuModelViewer(model)
view.nsteps=np.array([500,500,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=filename,strati=False,displacement_cmap = 'rainbow')
view.nsteps = np.array([500,500,50])
if(clip_on_dtm):
    colours = list(pd.DataFrame(data=proj.stratigraphic_column.column,columns=["name"]).merge(proj.stratigraphic_column.stratigraphicUnits[["name","colour"]], on="name")["colour"])
    colours.reverse()
    view.add_dtm(paint_with=lambda xyz: model.evaluate_model(xyz,scale=False), cmap=colours)
view.add_model()

view.lv.control.Range('alpha', label="Global Opacity")
view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)

# Show the control panel, including the viewer window
view.lv.control.show()
view.interactive()

t3 = time.time()

## Elapsed Time

In [ ]:
# Print element and total processing time
m2l_time = t1-t0
ls_time = t3-t2
total = t3-t0

m2l_string = f"{m2l_time} sec" if m2l_time < 60 else f"{m2l_time/60.0} min"
ls_string = f"{ls_time} sec" if ls_time < 60 else f"{ls_time/60.0} min"
total_string = f"{total} sec" if total < 60 else f"{total/60.0} min"
print(f"map2loop {m2l_string}, LoopStructural {ls_string}, Total {total_string}")